In [122]:
import re
import pickle
from copy import deepcopy
from collections import defaultdict

import requests
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy import sparse, stats
from scipy.special import expit as sigmoid
from tqdm import tqdm, trange

import torch
from torch import nn

In [19]:
import pickle
import os, json

import pandas as pd

DATA_RAW = 'data/raw'
tournaments = pickle.load(open(os.path.join(DATA_RAW, 'tournaments.pkl'), 'rb'))
results = pickle.load(open(os.path.join(DATA_RAW, 'results.pkl'), 'rb'))
players = pickle.load(open(os.path.join(DATA_RAW, 'players.pkl'), 'rb'))

In [20]:
result_preprocessed = defaultdict(list)
train_tournaments = []
test_tournaments = []

question_qty_train = 0
question_qty_test = 0

error_mask = []
for tour_id, tour_info in tournaments.items():
    tour_date = tour_info['dateStart'][:4] 
    if tour_date not in ('2019', '2020'):
        continue
        
    # отфильтруем команды без информации по правильно отвеченным вопросам
    # и у которых в поле 'mask' только цифры
    team_many = [team  for team in results[tour_id] if 'mask' in team.keys() and \
                 team['mask'] is not None and re.match("[0-9]+$", team['mask'])]
    if len(team_many) == 0:
        continue
    question_qty = max([len(team['mask']) for team in team_many])

    # отфильтруем команды с неполной статистикой по правильно отвеченным вопросам
    team_many = [team for team in team_many if len(team['mask']) == question_qty]
    if len(team_many) == 0:
        continue
    result_preprocessed[tour_id] = team_many
    if tour_date == '2019':
        train_tournaments.append((tour_id, tour_info['name']))
        question_qty_train += question_qty
    else:
        assert tour_date == '2020'
        test_tournaments.append((tour_id, tour_info['name']))
        question_qty_test += question_qty

# Baseline
Построим в качестве безлайна лог. рег. в задаче бинарной классификации - ответ ли участник на вопрос или нет, независимо от команды. В качестве фичей будут one-hot вектора всех пользователей и вопросов

In [21]:
player_many = set()

question_qty_all = 0
for tour_id, _ in train_tournaments:
    tournament_answers = []
    for team in result_preprocessed[tour_id]:
        for player in team['teamMembers']:
            player_many.add(player['player']['id'])
        

player_to_cat = {player:idx for idx, player in enumerate(player_many)}
cat_to_player = {idx:player for player, idx in player_to_cat.items()}


In [89]:
# Уберем из тестовой выборки игроков, которых не было в обучающей выборке
test_tournaments_preprocessed = []
tournaments_rating_true = []

for tour_id, _ in test_tournaments:
    team_many = result_preprocessed[tour_id]
    team_many_new = []
    teams_rating = []
    for team in team_many:
        team_new = deepcopy(team)
        team_new['teamMembers'] = []
        for player in team['teamMembers']:
            if player['player']['id'] in player_to_cat.keys():
                team_new['teamMembers'].append(player)
        if len(team_new['teamMembers']) > 0:
            team_many_new.append(team_new)
            team_answers = list(map(int, team_new['mask']))
            team_score = sum(team_answers)
            teams_rating.append(team_score)
            
    if len(team_many_new) > 0:
        test_tournaments_preprocessed.append(team_many_new)
        tournaments_rating_true.append(teams_rating)

In [116]:
player_idxs = []
question_idxs = []
team_ids = []
target = []

question_qty = 0
player_many_qty = len(player_to_cat)
for tour_id, _ in train_tournaments:
    question_tour_qty = len(result_preprocessed[tour_id][0]['mask'])
    for team in result_preprocessed[tour_id]:
        team_answers = list(map(int, team['mask']))
        for question in range(question_tour_qty):
            for player in team['teamMembers']:
                player_idxs.append(player_to_cat[player['player']['id']])
                question_idxs.append(player_many_qty + question_qty + question)
                target.append(team_answers[question])
                team_ids.append(team['team']['id'])
    question_qty += question_tour_qty

In [25]:
# обучающая выборка
# матрица размера:
# (кол-во уникальных игроков в выборке; кол-во уникальных игроков в выборке + кол-во уник. вопросов )

X = sparse.coo_matrix((
   [1] * len(player_idxs),
    (
        list(range(len(player_idxs))),
        player_idxs,
    )
),
    shape = (len(player_idxs), len(player_to_cat) + question_qty),
    dtype = np.int32,
)

X += sparse.coo_matrix((
   [1] * len(question_idxs),
    (
        list(range(len(player_idxs))),
        question_idxs,
    ), 
), 
    shape = (len(player_idxs), len(player_to_cat) + question_qty),
    dtype = np.int32,
)

y = np.array(target)
dim0, dim1 = X.shape

In [26]:
assert X.sum() == len(question_idxs) + len(player_idxs)
assert y.shape[0] == X.shape[0]

In [27]:
%%time

lr = LogisticRegression(tol=1e-1, solver='saga', C=10, n_jobs=-1)
lr.fit(X, y)

assert lr.coef_.shape[1] == len(player_to_cat) + question_qty

CPU times: user 56.9 s, sys: 344 ms, total: 57.3 s
Wall time: 57.3 s


In [28]:
lr_filename = 'lr_hw2.pkl'
pickle.dump(lr, open(lr_filename, 'wb'))

# lr = pickle.load(open(lr_filename, 'rb'))

In [29]:
rating_list = []
player_skill = lr.coef_[:len(player_to_cat)][0]

for player_idx, player_cat in player_to_cat.items():
    rating_list.append({
        'score': player_skill[player_cat],
        'id': player_idx,
        'name': players[player_idx]['name'] + ' ' + players[player_idx]['surname'],
    })
    
sorted_rating = sorted(rating_list, key=lambda x: x['score'], reverse=True)

In [30]:
def get_player_position(id):
    url = f'https://rating.chgk.info/api/players/{id}/rating/last'
    position = -1
    try:
        position = requests.get(url).json()['rating_position']
        position = int(position)
    except Exception as e:
        pass
    
    return position

In [31]:
df_rating = pd.DataFrame(sorted_rating)[:50]
df_rating['actual_position'] = df_rating['id'].apply(get_player_position)
df_rating.head(50)

,score,id,name,actual_position
0,4.438798,27403,Максим Руссо,5
1,4.292067,4270,Александра Брутер,6
2,4.242939,28751,Иван Семушин,3
3,4.211719,27822,Михаил Савченков,2
4,4.138163,30152,Артём Сорожкин,1
5,4.119870,30270,Сергей Спешков,4
6,4.105460,40411,Дмитрий Кудинов,-1
7,4.010716,38175,Максим Пилипенко,9946
8,3.984454,18036,Михаил Левандовский,8
9,3.975538,20691,Станислав Мереминский,38


In [32]:
top_50_count = df_rating[df_rating['actual_position'] <= 100].shape[0]

print(f'В топ-50 рейтинга {top_50_count} игроков из топ-100 реального рейтинга')

В топ-50 рейтинга модели попали 32 игроков из топ-100 реального рейтинга


Возьмем за рейтинг команды вероятность правильного ответа на вопрос хотя бы одним участником команды: $P(team=1) = 1 - \prod P(player=0)$

Сложность вопросов заранее неизвестна, поэтому просто занулим коэфициенты у лог. рег. с прошлого шага, отвечающие за сложности вопросов

In [85]:
def predict_tournaments(model, tournaments, member_to_idx):

    tournaments_rating_pred = []
    for torunament in tqdm(tournaments, position=0, leave=False):
        tournament_questions_count = len(torunament[0]['mask'])
        preds = []
        for team in torunament:
            memeber_idxs = [member_to_idx[member['player']['id']] for member in team['teamMembers']]
            members_count = len(memeber_idxs)

            X = sparse.lil_matrix((members_count, dim1), dtype=int)
            X[range(len(memeber_idxs)), memeber_idxs] = 1

            fail_probas = model.predict_proba(X)[:, 0]
            team_proba = 1 - fail_probas.prod() 
            preds.append(team_proba)

        tournaments_rating_pred.append(preds)
    return tournaments_rating_pred

def get_corr(tournaments_rating_true, tournaments_rating_pred):
    spearmanr_corrs = []
    kendall_corrs = []
    for i in range(len(tournaments_rating_true)):
        spearman = stats.spearmanr(tournaments_rating_true[i], tournaments_rating_pred[i]).correlation
        kendall = stats.kendalltau(tournaments_rating_true[i], tournaments_rating_pred[i]).correlation
        spearmanr_corrs.append(spearman)
        kendall_corrs.append(kendall)

    print(f'Корреляция Спирмена: {np.mean(spearmanr_corrs):.4f}')
    print(f'Корреляция Кендалла: {np.mean(kendall_corrs):.4f}')

In [ ]:
tournaments_rating_true = []

for tournament in tournaments_val:
    teams_test = []
    teams_rating = []
    for team in tournament['teams']:
  
        memebers = [member for member in team['members'] if member in member_to_idx.keys()]
        team_test = copy(team)
        team_test['members'] = memebers
        
        if len(memebers) > 0:
            teams_test.append(team_test)
            team_answers = list(map(int, team['mask']))
            team_score = sum(team_answers)
            teams_rating.append(team_score)

In [90]:
tournaments_rating_pred = predict_tournaments(
    model=lr, 
    tournaments=test_tournaments_preprocessed,
    member_to_idx=player_to_cat, 
)
assert len(tournaments_rating_pred) == len(tournaments_rating_true)

get_corr(tournaments_rating_true, tournaments_rating_pred)

Корреляция Спирмена: 0.7985
Корреляция Кендалла: 0.6431


# EM-алгоритм

Возьмем в качестве вектора скрытых переменных вероятность правильного ответа игроком на вопрос при условии команды: $z = P(member=1|team)$

Упрощающее предположение - если команда не ответила на вопрос, то никто в команде не ответил на вопрос: $P(member=1|team=0) = 0$

E-шаг: предсказываем вероятности ответа на вопрос игрока при условии команды: $P(member=1|team=1) = \frac{P(member=1 \cap team=1)}{P(team=1)} = \frac{P(team=1 | member=1) P(member=1)}{P(team=1)} = \frac{P(member=1)}{P(team=1)}$
    
M-шаг: максимизируем правдоподобие. Обучаем модель на вероятностях с E-шага



In [102]:
from sklearn.linear_model import LinearRegression

class LogRegSoftLabel:
    """
    logistic regression on soft labels
    """
    def __init__(self):
        pass
    
    def fit(self, X, y, *args, **kwargs):
        y_tr = np.clip(y, 1e-8, 1 - 1e-8)   # numerical stability
        inv_sig_y = np.log(y_tr / (1.0 - y_tr))  # transform to log-odds-ratio space
        
        self._lr = LinearRegression(*args, **kwargs)
        self._lr.fit(X, inv_sig_y)
        
    def predict_proba(self, X):
        preds = sigmoid(self._lr.predict(X))
        return np.hstack(((1 - preds).reshape(-1, 1), preds.reshape(-1, 1)))

In [103]:
%%time
model = LogRegSoftLabel()
model.fit(X, y)

In [105]:
tournaments_rating_pred = predict_tournaments(
    model=model, 
    tournaments=test_tournaments_preprocessed,
    member_to_idx=player_to_cat, 
)
get_corr(tournaments_rating_true, tournaments_rating_pred)

Корреляция Спирмена: 0.7939
Корреляция Кендалла: 0.6364


In [119]:
for _ in trange(15):
    
    # E-шаг
    preds = model.predict_proba(X)
    
    # Вероятность ответа игрока при условии команды
    df_team = pd.DataFrame({'team': team_ids, 'question': question_idxs, 'fail_pred': preds[:, 0], 'success_pred': preds[:, 1]})
    df_team_pis = df_team.groupby(['team', 'question']).agg({'fail_pred': 'prod'}).reset_index()
    df_team_pis['team_success_pred'] = 1 - df_team_pis['fail_pred']
    df_team_pis.drop(columns=['fail_pred'], inplace=True)
    df_team = pd.merge(df_team, df_team_pis, left_on=['team', 'question'],  right_on=['team', 'question'])
    z = (df_team['success_pred'] / df_team['team_success_pred']).clip(0, 1)
    
    # Если команда не ответила на вопрос, то никто из команды на него не ответил
    z[y == 0] = 0
    
    # M-шаг
    model.fit(X, z)

    tournaments_rating_pred = predict_tournaments(
        model=model, 
        tournaments=test_tournaments_preprocessed,
        member_to_idx=player_to_cat, 
    )
    get_corr(tournaments_rating_true, tournaments_rating_pred)

  7%|▋         | 1/15 [02:15<31:39, 135.69s/it]  

Корреляция Спирмена: 0.7805
Корреляция Кендалла: 0.6233


 13%|█▎        | 2/15 [04:30<29:19, 135.33s/it]  

Корреляция Спирмена: 0.7896
Корреляция Кендалла: 0.6307


 20%|██        | 3/15 [06:46<27:06, 135.52s/it]  

...

Корреляция Спирмена: 0.7963
Корреляция Кендалла: 0.6389


 93%|█████████▎| 14/15 [31:59<02:18, 138.69s/it] 

Корреляция Спирмена: 0.7962
Корреляция Кендалла: 0.6389


100%|██████████| 15/15 [34:19<00:00, 137.29s/it] 

Корреляция Спирмена: 0.7966
Корреляция Кендалла: 0.6391
